In [1]:
import pandas as pd
import re
import spacy
from tqdm import tqdm

tqdm.pandas()

# IPython magic commands
%load_ext autoreload
%autoreload 2

In [2]:
# Load Reuters dataset
reuters_df = pd.read_csv("data/processed/reuters-test.csv")

# Initialize SpaCy for Named Entity Recognition (NER)
nlp = spacy.load("en_core_web_sm")  # English small model

# Regex pattern to detect tickers (examples: $AAPL or AAPL)
ticker_pattern = re.compile(r"\$?[A-Z]{1,5}\b")

In [3]:
def extract_entities(title):
    """
    Extracts company names or tickers from a title.
    Returns a unique list of entities.
    """
    # Apply SpaCy NER to detect organization names (ORG)
    doc = nlp(title)
    orgs = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    
    # Find tickers in the text using regex
    tickers = ticker_pattern.findall(title)
    
    # Clean tickers by removing $ if present
    tickers = [t.replace("$", "") for t in tickers]
    
    # Combine NER-detected names and tickers, remove duplicates
    entities = list(set(orgs + tickers))
    return entities

In [4]:
# Apply entity extraction to the 'title' column
reuters_df['entities'] = reuters_df['title'].apply(extract_entities)

# Save the results to a new CSV file
reuters_df.to_csv("data/processed/reuters_entities.csv", index=False)

print("Extraction completed! Example output:")
print(reuters_df[['title', 'entities']].head())

Extraction completed! Example output:
                                               title entities
0  WITNESS: Back to work with a broken neck and u...  [TNESS]
1  China eyes investing in private equity, hedge ...       []
2         Toys "R" Us stops sales of China-made bibs      [R]
3  Philippines says captures rebel camp in bloody...       []
4     Afghans search for kidnapped German aid worker       []


In [5]:
reuters_df[['title', 'entities']]

,title,entities
0,WITNESS: Back to work with a broken neck and u...,[TNESS]
1,"China eyes investing in private equity, hedge ...",[]
2,"Toys ""R"" Us stops sales of China-made bibs",[R]
3,Philippines says captures rebel camp in bloody...,[]
4,Afghans search for kidnapped German aid worker,[]
...,...,...
9891,Billionaire investors turn bearish as U.S. sto...,"[U, S]"
9892,"Berkshire takes bigger bite of Apple, pares Wa...","[Wal-Mart, Apple]"
9893,Soros Fund Management slashes gold stake in se...,[Soros Fund Management]
9894,"Beltre, Rangers slam A's","[Rangers, A]"


In [19]:
import json5
from mistralai import Mistral
import time

# Initialize Mistral client (make sure you set your API key: export MISTRAL_API_KEY=xxxx)
client = Mistral(api_key="EKQmi23qrX0UWjRLw0RpH2NiJG0m4tK8")

def analyze_headline(headline: str):
    """
    Send a news headline to Mistral LLM and return structured JSON output.
    If no relevant company/industry is found, 'relevant_company_details' is empty.
    """

    # Prompt carefully designed to avoid hallucinations
    prompt = f"""
<headline>
{headline}
</headline>

<instructions>
You are given a news headline.  
Your task is to:
1. Identify relevant companies or industries explicitly mentioned or clearly implied.  
   - If none are relevant, return an empty dictionary: "relevant_company_details": {{}}  
2. For each identified company or industry, provide:
   - ticker if available, else "NA"  
   - sentiment (positive / negative / neutral)  
   - reasoning behind the sentiment, based only on the headline  
3. Suggest a few keywords that summarize the main topic.  
4. Exclude news publishers, governments, or organizations not acting as companies/industries.

Return your answer strictly in this JSON format:
{{
  "article_keywords": ["...", "..."],
  "relevant_company_details": {{
    "company_or_industry_name": {{
      "ticker": "ticker/NA",
      "sentiment": "positive/neutral/negative",
      "sentiment_reasoning": "..."
    }}
  }}
}}
</instructions>
"""
    time.sleep(1)  # To avoid rate limits

    try:
        # Call Mistral API
        response = client.chat.complete(
            model="mistral-large-latest",
            messages=[{"role": "user", "content": prompt}]
        )

        raw_output = response.choices[0].message.content
        
        if raw_output.startswith("```json"):
            raw_output = raw_output.replace("```json", "").replace("```", "").strip()

        # Try to parse the model output as JSON
        result = json5.loads(raw_output)

        # Ensure consistent format
        if "article_keywords" not in result or not isinstance(result["article_keywords"], list):
            print(f"⚠️ Warning: 'article_keywords' missing or invalid in response for headline '{headline}'. Setting to empty list.")
            result["article_keywords"] = []
        if "relevant_company_details" not in result or not isinstance(result["relevant_company_details"], dict):
            print(f"⚠️ Warning: 'relevant_company_details' missing or invalid in response for headline '{headline}'. Setting to empty dict.")
            result["relevant_company_details"] = {}

        return result

    except Exception as e:
        print(f"⚠️ Error with headline '{headline}': {e}")
        # Return safe fallback structure
        return {
            "article_keywords": [],
            "relevant_company_details": {}
        }

# Example DataFrame with headlines
df = pd.DataFrame({
    "title": [
        "Tesla faces production delays in Berlin Gigafactory",
        "Oil prices drop as OPEC signals higher output",
        "S.Africa truck, taxi collision kills 22"
    ]
})

df = reuters_df.loc[60:100].copy()

# Apply LLM analysis to each headline
df["llm_result"] = df["title"].apply(analyze_headline)

# Split out useful parts of the JSON into separate columns
df["llm_keywords"] = df["llm_result"].apply(lambda x: x.get("article_keywords"))
df["llm_company_details"] = df["llm_result"].apply(lambda x: x.get("relevant_company_details"))

# Display final DataFrame
df

⚠️ Error with headline 'FACTBOX: Security developments in Iraq': API error occurred: Status 429
{"object":"error","message":"Service tier capacity exceeded for this model.","type":"service_tier_capacity_exceeded","param":null,"code":"3505"}


,ts,title,href,ts_parsed,entities,llm_result,llm_keywords,llm_company_details
60,20070819 06:39 AM EDT,Kazakh vote below OSCE standards but step forward,http://www.reuters.com/article/worldNews/idUSL...,2007-08-19 10:39:00+00:00,"[OSCE, Kazakh]","{'article_keywords': ['Kazakhstan elections', ...","[Kazakhstan elections, OSCE standards, democra...",{}
61,20070819 06:40 AM EDT,Hyundai to raise Russia kit exports by 60 pct ...,http://www.reuters.com/article/tnBasicIndustri...,2007-08-19 10:40:00+00:00,[Hyundai],"{'article_keywords': ['Hyundai', 'Russia', 'ki...","[Hyundai, Russia, kit exports, automotive indu...","{'Hyundai': {'ticker': 'NA', 'sentiment': 'pos..."
62,20070819 06:48 AM EDT,Polish PM says opposition too dependent on Ger...,http://www.reuters.com/article/worldNews/idUSL...,2007-08-19 10:48:00+00:00,[PM],{'article_keywords': ['Poland-Germany relation...,"[Poland-Germany relations, political oppositio...",{}
63,20070819 06:51 AM EDT,Bangladesh crackdown gives hope to exiled writer,http://www.reuters.com/article/reutersEdge/idU...,2007-08-19 10:51:00+00:00,[],"{'article_keywords': ['Bangladesh', 'crackdown...","[Bangladesh, crackdown, exiled writer, freedom...",{}
64,20070819 06:56 AM EDT,Maldives President wins landmark poll,http://www.reuters.com/article/latestCrisis/id...,2007-08-19 10:56:00+00:00,[Maldives],"{'article_keywords': ['Maldives', 'President',...","[Maldives, President, election, landmark poll,...",{}
65,20070819 06:57 AM EDT,Endgame looms as nuclear deal strains Indian g...,http://www.reuters.com/article/newsOne/idUSDEL...,2007-08-19 10:57:00+00:00,[],"{'article_keywords': ['nuclear deal', 'India',...","[nuclear deal, India, government strain, endga...",{'Nuclear Energy Industry (India)': {'ticker':...
66,20070819 07:02 AM EDT,Afghan TV airs hostage video,http://www.reuters.com/news/video/videoStory?s...,2007-08-19 11:02:00+00:00,[TV],"{'article_keywords': ['Afghanistan', 'hostage'...","[Afghanistan, hostage, video, TV broadcast, me...",{}
67,20070819 07:02 AM EDT,"FACTBOX-Security developments in Iraq, Aug 19",http://www.reuters.com/article/middleeastCrisi...,2007-08-19 11:02:00+00:00,[CTBOX],"{'article_keywords': ['Iraq', 'security develo...","[Iraq, security developments, conflict, geopol...",{}
68,20070819 07:07 AM EDT,"More rain, diseases worsen South Asia floods",http://www.reuters.com/article/asiaCrisis/idUS...,2007-08-19 11:07:00+00:00,[],"{'article_keywords': ['South Asia floods', 'he...","[South Asia floods, heavy rainfall, waterborne...",{}
69,20070819 07:07 AM EDT,"More rain, diseases worsen South Asia floods",http://www.reuters.com/article/homepageCrisis/...,2007-08-19 11:07:00+00:00,[],"{'article_keywords': ['South Asia floods', 'he...","[South Asia floods, heavy rainfall, disease ou...",{}


In [26]:
from pyspark.sql import SparkSession

# Créer une session Spark
spark = SparkSession.builder \
    .appName("Analyse FNSPID") \
    .getOrCreate()

# Lire le CSV (Spark lit par partitions, pas tout en RAM)
df = spark.read.csv("data/raw/All_external.csv", header=True, inferSchema=True)

# Afficher les 50 premières lignes
df.show(50, truncate=False)  # truncate=False pour ne pas couper le texte dans les colonnes


+-----------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------+------+-------+-----------+------------+----------------+---------------+
|Date                   |Article_title                                                                                                                                                                                                                                                  |Stock_symbol|Url                                                                                                                                                 |Publisher       

In [6]:
financial_news_df = pd.read_csv("data/raw/daily_financial_news/analyst_ratings_processed.csv", usecols=['title', 'date', 'stock'], parse_dates=['date'])

In [7]:
financial_news_df.head()

,title,date,stock
0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A
3,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A
4,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A


In [8]:
financial_news_df["stock"].isna().sum()

2578

In [9]:
financial_news_df["stock"].dropna().nunique()

6192

In [10]:
financial_news_df["stock"].dropna().unique()

array(['A', 'AAMC', 'AAME', ..., 'ZU', 'ZUMZ', 'ZX'], dtype=object)

In [12]:
import wrds

# Connexion (WRDS demande un compte)
db = wrds.Connection(wrds_username='matthiasw')


Loading library list...
Done


In [ ]:
# Table des noms des actions
tickers_df = db.get_table(library='crsp', table='stocknames')


In [4]:
crsp_data = pd.read_csv("data/processed/all_stocks_us_with_vol_flag.csv", parse_dates=['date'])

In [5]:
crsp_data.head()

,permno,date,ret,prc,vol,comnam,shrcd,exchcd,ticker,on_rdq,vol_missing_flag
0,11194,2001-01-02,0.000000,8.7500,700.0,MINUTEMAN INTERNATIONAL INC,11,3,MMAN,0,0
1,11187,2001-01-02,0.006479,29.1250,18000.0,MERIDIAN INSURANCE GROUP INC,11,3,MIGI,0,0
2,39328,2001-01-02,0.052632,3.7500,5200.0,GOLDEN ENTERPRISES INC,11,3,GLDC,0,0
3,12022,2001-01-02,-0.013158,23.4375,8179.0,GENLYTE GROUP INC,11,3,GLYT,0,0
4,11705,2001-01-02,-0.006342,14.6875,3625.0,LAUREL CAPITAL GROUP INC,11,3,LARL,0,0


In [31]:
# 1. Charger stocknames depuis CRSP
stocknames = db.raw_sql("""
    SELECT permco, ticker
    FROM crsp.stocknames
""")
stocknames['ticker'] = stocknames['ticker'].str.upper().str.strip()
stocknames.drop_duplicates(subset="ticker", inplace=True)

In [32]:
stocknames.head()

,permco,ticker
0,7952,OMFGA
1,7953,GFGC
2,7953,EWST
4,7953,EGAS
7,7954,MBNC


In [34]:
financial_news_df = pd.read_csv("data/raw/All_external.csv", parse_dates=['Date'], nrows=500000)
financial_news_df.rename(columns={"Date": "news_date", "Stock_symbol": "ticker"}, inplace=True)
financial_news_df['ticker'] = financial_news_df['ticker'].str.upper().str.strip()
financial_news_df['news_date'] = pd.to_datetime(financial_news_df['news_date'])

In [35]:
financial_news_df.head()

,news_date,Article_title,ticker,Url,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary
0,2020-06-05 06:30:54+00:00,Stocks That Hit 52-Week Highs On Friday,A,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-06-03 06:45:20+00:00,Stocks That Hit 52-Week Highs On Wednesday,A,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-05-26 00:30:07+00:00,71 Biggest Movers From Friday,A,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-05-22 08:45:06+00:00,46 Stocks Moving In Friday's Mid-Day Session,A,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-05-22 07:38:59+00:00,B of A Securities Maintains Neutral on Agilent...,A,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
merged = financial_news_df.merge(stocknames, on="ticker", how="left")

In [38]:
merged.head()

,news_date,Article_title,ticker,Url,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary,permco
0,2020-06-05 06:30:54+00:00,Stocks That Hit 52-Week Highs On Friday,A,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,NaN,NaN,NaN,NaN,NaN,NaN,22196
1,2020-06-03 06:45:20+00:00,Stocks That Hit 52-Week Highs On Wednesday,A,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,NaN,NaN,NaN,NaN,NaN,NaN,22196
2,2020-05-26 00:30:07+00:00,71 Biggest Movers From Friday,A,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,NaN,NaN,NaN,NaN,NaN,NaN,22196
3,2020-05-22 08:45:06+00:00,46 Stocks Moving In Friday's Mid-Day Session,A,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,NaN,NaN,NaN,NaN,NaN,NaN,22196
4,2020-05-22 07:38:59+00:00,B of A Securities Maintains Neutral on Agilent...,A,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,NaN,NaN,NaN,NaN,NaN,NaN,22196


In [2]:
financial_news_df = pd.read_csv("data/raw/All_external.csv", parse_dates=['Date'], nrows=500000)

In [3]:
from src.merge_crsp_fnspid import add_permco_to_news
df_with_permco = add_permco_to_news(financial_news_df)

ModuleNotFoundError: No module named 'dotenv'